In [67]:
from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
    Huber,
    TheilSenRegressor,
    RANSACRegressor
)
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    KFold
)
from sklearn.metrics import (
    mean_absolute_error as MAE,
    root_mean_squared_error as RMSE,
    mean_squared_error as MSE,
    median_absolute_error as MdAE,
    r2_score as R2
)
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import pandas as pd
import numpy as np

In [77]:
df = pd.read_csv("data_for_model.csv", delimiter=';', header=None)
df.columns = ['ID', 'TerritoryName', 'TerritoryGroup', 'CountryRegionCode', 'ShipMethodName', 'TotalUnits', 'Avg_StdCost', 'TotalDue']
df


,ID,TerritoryName,TerritoryGroup,CountryRegionCode,ShipMethodName,TotalUnits,Avg_StdCost,TotalDue
0,43659,Southeast,North America,US,CARGO TRANSPORT 5,26,1120.2741,23153.2339
1,43660,Southeast,North America,US,CARGO TRANSPORT 5,2,685.7074,1457.3288
2,43661,Canada,North America,CA,CARGO TRANSPORT 5,38,883.0531,36865.8012
3,43662,Canada,North America,CA,CARGO TRANSPORT 5,54,619.8741,32474.9324
4,43663,Southwest,North America,US,CARGO TRANSPORT 5,1,486.7066,472.3108
...,...,...,...,...,...,...,...,...
31460,75119,Northwest,North America,US,XRQ - TRUCK GROUND,3,5.2709,46.7194
31461,75120,Canada,North America,CA,XRQ - TRUCK GROUND,3,18.9050,93.8808
31462,75121,Canada,North America,CA,XRQ - TRUCK GROUND,3,9.3475,82.8529
31463,75122,Canada,North America,CA,XRQ - TRUCK GROUND,2,7.5714,34.2219


In [65]:
df['TerritoryName'].unique()
df['TerritoryName'] = df['TerritoryName'].replace({
    'Southeast':0,
    'Canada':1,
    'Southwest':2,
    'Northwest':3,
    'Central':4,
    'Northeast':5,
    'France':6,
    'Australia':7,
    'United Kingdom':8,
    'Germany':9
})

In [78]:
df = pd.get_dummies(df, columns = ['TerritoryName', 'TerritoryGroup', 'CountryRegionCode', 'ShipMethodName'])


In [79]:
df

,ID,TotalUnits,Avg_StdCost,TotalDue,TerritoryName_Australia,TerritoryName_Canada,TerritoryName_Central,TerritoryName_France,TerritoryName_Germany,TerritoryName_Northeast,...,TerritoryGroup_North America,TerritoryGroup_Pacific,CountryRegionCode_AU,CountryRegionCode_CA,CountryRegionCode_DE,CountryRegionCode_FR,CountryRegionCode_GB,CountryRegionCode_US,ShipMethodName_CARGO TRANSPORT 5,ShipMethodName_XRQ - TRUCK GROUND
0,43659,26,1120.2741,23153.2339,False,False,False,False,False,False,...,True,False,False,False,False,False,False,True,True,False
1,43660,2,685.7074,1457.3288,False,False,False,False,False,False,...,True,False,False,False,False,False,False,True,True,False
2,43661,38,883.0531,36865.8012,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,True,False
3,43662,54,619.8741,32474.9324,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,True,False
4,43663,1,486.7066,472.3108,False,False,False,False,False,False,...,True,False,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31460,75119,3,5.2709,46.7194,False,False,False,False,False,False,...,True,False,False,False,False,False,False,True,False,True
31461,75120,3,18.9050,93.8808,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,False,True
31462,75121,3,9.3475,82.8529,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,False,True
31463,75122,2,7.5714,34.2219,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,False,True


In [80]:
X = df.drop(columns=['ID', 'TotalDue'])
y = df['TotalDue']

X

,TotalUnits,Avg_StdCost,TerritoryName_Australia,TerritoryName_Canada,TerritoryName_Central,TerritoryName_France,TerritoryName_Germany,TerritoryName_Northeast,TerritoryName_Northwest,TerritoryName_Southeast,...,TerritoryGroup_North America,TerritoryGroup_Pacific,CountryRegionCode_AU,CountryRegionCode_CA,CountryRegionCode_DE,CountryRegionCode_FR,CountryRegionCode_GB,CountryRegionCode_US,ShipMethodName_CARGO TRANSPORT 5,ShipMethodName_XRQ - TRUCK GROUND
0,26,1120.2741,False,False,False,False,False,False,False,True,...,True,False,False,False,False,False,False,True,True,False
1,2,685.7074,False,False,False,False,False,False,False,True,...,True,False,False,False,False,False,False,True,True,False
2,38,883.0531,False,True,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,True,False
3,54,619.8741,False,True,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,True,False
4,1,486.7066,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31460,3,5.2709,False,False,False,False,False,False,True,False,...,True,False,False,False,False,False,False,True,False,True
31461,3,18.9050,False,True,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,True
31462,3,9.3475,False,True,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,True
31463,2,7.5714,False,True,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,True


In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

In [90]:
df['TotalDue'].mean()

3915.9951093564277

In [101]:
avg_model = pd.Series([3916] * 9440)

In [96]:
y_test.shape

(9440,)

In [83]:
linear = LinearRegression()

linear.fit(X_train, y_train)
y_pred = linear.predict(X_test)
MAE(y_test, y_pred), RMSE(y_test, y_pred)

(1895.8360173231658, 5830.022432119296)

In [95]:
ridge = LinearRegression()

ridge.fit(X_train, y_train)
y_predr = ridge.predict(X_test)
MAE(y_test, y_predr), RMSE(y_test, y_predr)

(1895.8360173231658, 5830.022432119296)

In [103]:
MAE(y_test, avg_model), RMSE(y_test, avg_model)

(5143.885148993643, 12788.72222148543)